In [2]:
import os
import glob
import shutil

import paramiko
from scp import SCPClient
import posixpath

In [ ]:
msh_h5_files = glob.glob(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh' + '/*.msh.h5')
msh_h5_files = [file.replace(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh', '') for file in msh_h5_files]
msh_h5_files = [file.replace("\\", "") for file in msh_h5_files]
msh_h5_files = [file.replace(".msh.h5", "") for file in msh_h5_files]
msh_h5_files = pd.DataFrame(msh_h5_files, columns=['name'])
msh_h5_files.to_excel(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh\240913_msh_h5_files.xlsx')

rows_to_move = list(np.arange(7,15))
new_index = rows_to_move + [index for index in msh_h5_files.index if index not in rows_to_move]
msh_h5_files = msh_h5_files.reindex(new_index)

rows_to_move = [7,13]
new_index = rows_to_move + [index for index in msh_h5_files.index if index not in rows_to_move]
msh_h5_files = msh_h5_files.reindex(new_index)

msh_h5_files.reset_index(drop=True,inplace=True)

msh_h5_files['code'] = ['A1_M1','A2_M1','A3_M1','A3_M2','A3_M3','A3_M4','A3_M5','A4_M1','A5_M1','A6_M1','A7_M1','A8_M1','A9_M1','A10_M1','A11_M1']

new_names = ['V4.5NI_s_plus_TEST',
 'V4.5NI_s_oldM',
 'V4.5NI_s_r60',
 'V4.5NI_s_r60',
 'V4.5NI_s_r60',
 'V4.5NI_s_r60',
 'V4.5NI_s_r60',
 'V4.5NI_s_wHB_r60',
 'V4.5F2014_s_wHB_r60',
 'V4.5F2014_s_wHB_r120',
 'V4.5F2020_s_wHB_r60',
 'V4.5F5020_s_wHB_r60',
 'V4.5F5020_s_wHB_r120',
 'V4.5F5030_s_wHB_r60',
 'V4.5F5040_s_wHB_r60',]

msh_h5_files['new_name'] = [msh_h5_files.loc[i, 'code'] + '__' + new_names[i] for i in range(len(msh_h5_files.index))]

In [ ]:
for index, row in msh_h5_files.iterrows():
    old_name = os.path.join(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh', row['name'] + '.msh.h5')
    new_name = os.path.join(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh', row['new_name'] + '.msh.h5')
    os.rename(old_name, new_name)

In [5]:
def copy_folder_with_exclusions(src, dest, excluded_extensions):
    # Make sure the destination folder exists
    if not os.path.exists(dest):
        os.makedirs(dest)
    
    # Walk through the source directory
    for root, dirs, files in os.walk(src):
        # Create the corresponding directory in the destination
        relative_path = os.path.relpath(root, src)
        dest_dir = os.path.join(dest, relative_path)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
        
        # Filter and copy files, excluding specific extensions
        for file in files:
            if not file.endswith(tuple(excluded_extensions)):
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest_dir, file)
                shutil.copy2(src_file, dest_file)

In [9]:
def move_files_with_extension(source_dir, dest_dir, extension):
    # Walk through the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(extension):
                # Create the destination subfolder structure
                relative_path = os.path.relpath(root, source_dir)
                dest_subdir = os.path.join(dest_dir, relative_path)
                os.makedirs(dest_subdir, exist_ok=True)
                # Move the file to the destination directory
                source_file = os.path.join(root, file)
                dest_file = os.path.join(dest_subdir, file)
                shutil.move(source_file, dest_file)
                print(f"Moved: {source_file} -> {dest_file}")

In [ ]:
main_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI'
Mesh_fp = os.path.join(main_fp, 'Mesh')

msh_h5_files = glob.glob(Mesh_fp + '/*.msh.h5')
msh_h5_files = [file for file in msh_h5_files if (("M1" in file) & ("oldM" not in file) & ("plus_TEST" not in file) & ("_r120" not in file))]
# msh_h5_files = [file for file in msh_h5_files if (("M1" in file) & ("oldM" not in file) & ("plus_TEST" not in file) & ("_r60" not in file))]

msh_h5_files = [os.path.basename(file_path) for file_path in msh_h5_files]
msh_h5_files = sorted(msh_h5_files, key=lambda x: int(x[1:x.index('_M')]))
msh_h5_files = [file_name.replace(".msh.h5", "") for file_name in msh_h5_files]
msh_h5_files = [msh_h5_files[2]] + msh_h5_files[3:7] + msh_h5_files[0:2]
# msh_h5_files.append('A10_M1__V4.5F5030_s_wHB_r60')
msh_h5_files

In [1]:
msh_h5_files = [
                 'A5_M1__V4.5F2014_s_wHB_r60',
                 'A7_M1__V4.5F2020_s_wHB_r60',
                ]

In [7]:
# Usage

for i, caoMeshName in enumerate(msh_h5_files):

    caoMeshCode = caoMeshName.split('__')[0]

    source_folder = os.path.join(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults', str(caoMeshCode))
    destination_folder = os.path.join(r'H:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel\CFD\Fluent_GMI\SimulationResults', str(caoMeshCode))
    
    exclusions = ['.cas.h5', '.dat.h5']
    copy_folder_with_exclusions(source_folder, destination_folder, exclusions)

In [8]:
source_folder = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\V4.5_1channel\SimulationResults\parametric_laminar_try0'
destination_folder = r'H:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\CFD\V4.5_1channel_SR'

exclusions = ['.cas.h5', '.dat.h5']
copy_folder_with_exclusions(source_folder, destination_folder, exclusions)

In [ ]:
# Example usage
source_folder = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
destination_folder = r'E:\DualSun\SEAGULL-3000\Fluent_GMI\tests'

file_extension = '.h5'  # Change to the desired file extension
move_files_with_extension(source_folder, destination_folder, file_extension)

## SSH

In [3]:
# Function to establish an SSH connection
def create_ssh_client(hostname, port, username, password):
    ssh = paramiko.SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, port=port, username=username, password=password)
    return ssh

# Recursive function to copy files and folders via SSH, excluding .h5 files
def copy_files_recursively(sftp, scp, remote_folder, local_folder):
    try:
        os.makedirs(local_folder, exist_ok=True)  # Create local folder if it doesn't exist
        items = sftp.listdir_attr(remote_folder)  # List directory contents

        for item in items:
            remote_path = posixpath.join(remote_folder, item.filename)  # Ensure POSIX (Linux) path
            local_path = os.path.join(local_folder, item.filename)  # Windows path uses os.path.join

            if item.st_mode & 0o40000:  # If it's a directory
                print(f"Entering directory {remote_path}")
                copy_files_recursively(sftp, scp, remote_path, local_path)
            else:  # If it's a file
                if not item.filename.endswith('.h5'):
                    print(f"Copying file {remote_path} to {local_path}")
                    scp.get(remote_path, local_path)
                else:
                    print(f"Skipping file {remote_path} (excluded by extension)")
    except Exception as e:
        print(f"Error while copying files: {e}")

# Main function to establish SSH connection and start file copy
def copy_files_via_ssh(hostname, port, username, password, remote_folder, local_folder):
    ssh = create_ssh_client(hostname, port, username, password)
    
    # Create SCP client
    with SCPClient(ssh.get_transport()) as scp:
        # Use SFTP to list files and directories on the remote directory
        sftp = ssh.open_sftp()
        try:
            copy_files_recursively(sftp, scp, remote_folder, local_folder)
        except Exception as e:
            print(f"Error during file copy: {e}")
        finally:
            sftp.close()
            ssh.close()

In [ ]:
# Define your SSH and folder details here
hostname = 'gnode03'
port = 22
username = 'delachaux'
password = 'finsonSPRING4?'  # Use SSH key for better security if possible
# remote_folder = '/usrtmp/delachaux/ANSYS Fluent Projects/Hydraulics/NUMAIRO_motif/SimulationResults/abaque1_laminar_try0'
remote_folder = '/usrtmp/delachaux/ANSYS Fluent Projects/Hydraulics/V4.5_1channel/SimulationResults/V4.5_1chMid_M1/parametric_laminar_try0'
local_folder = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\V4.5_1channel\SimulationResults\V4.5_1chMid_M1'
# local_folder = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\ANSYS Fluent Projects\Hydraulics\V4_motif\SimulationResults\V4.6_1MPE_M2\abaque1_laminar_try0'

# Start copying files, excluding those with .h5 extension
copy_files_via_ssh(hostname, port, username, password, remote_folder, local_folder)